In [3]:
import tensorflow as tf
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from IPython import display

# variable initialization functions
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [127]:
fisher_blocks = True
n_layers = 2
lam = 1/100
input_dim = 784
output_dim = 10
tf.reset_default_graph()
x = tf.placeholder(tf.float32, shape=[None, 784], name='input')
y_ = tf.placeholder(tf.float32, shape=[None, 10], name='output')
#in_dim = int(x.get_shape()[1]) # 784 for MNIST
#out_dim = int(y_.get_shape()[1]) # 10 for MNIST
weights = []
biases = []
hiddens = []
F_accum_weights = []
F_accum_biases = []

weights.append(weight_variable([784,600]))
biases.append(bias_variable([600]))    
hiddens.append(tf.nn.relu(tf.add(tf.matmul(x,weights[0]), biases[0])))

layer_loss = []
#layer_param = np.linspace(1,100, n_layers)


for i in range(1, n_layers):
    weights.append(weight_variable([600,600]))
    biases.append(bias_variable([600]))
    hiddens.append(tf.nn.relu(tf.add(tf.matmul(hiddens[i-1],weights[i]), biases[i]), name='hidden'+str(i)))
    print('Layer done', i)

if fisher_blocks:    
    for i in range(2, n_layers):
        with tf.name_scope('layer_fisher'):
            #Get Gradients of outputs wrt. weights of the layer
            ders_weights = tf.gradients(-tf.log(hiddens[i]), weights[i])
            ders_biases = tf.gradients(-tf.log(hiddens[i]), biases[i])
            #Square the gradients
            F_accum_weights.append([tf.square(ders_weights)])
            F_accum_biases.append([tf.square(ders_biases)])
            #Square the difference between next layer weights and the current layer weights.
            weight_sqrs = tf.square(weights[i] - weights[i-1])
            bias_sqrs = tf.square(biases[i] - biases[i-1])
            #Grad * Weight Square difference
            weight_leftout = tf.multiply(F_accum_weights[i-2], weight_sqrs)
            bias_leftout = tf.multiply(F_accum_biases[i-2], bias_sqrs)
            #Convert that to a number
            layer_loss.append((lam/2) * tf.reduce_sum(weight_leftout + bias_leftout))#tf.multiply(F_accum[i-1],tf.square(var_list[i] - var_list[i-1]))))
        print('Loss Graph done', i)

weights.append(weight_variable([600,600]))
biases.append(bias_variable([600])) 
hiddens.append(tf.nn.relu(tf.add(tf.matmul(hiddens[-1],weights[-1]), biases[-1]), name='hidden'+str(i)))
weights.append(weight_variable([600,10]))
biases.append(bias_variable([10]))
#hiddens.append(tf.nn.relu(tf.add(tf.matmul(hiddens[-1],weights[-1]), biases[-1]), name='hidden'+str(i)))




fisher_loss = 0
if fisher_blocks:
    with tf.name_scope('fisher_loss'):
        fisher_loss += tf.reduce_mean(tf.convert_to_tensor(layer_loss))
        tf.summary.scalar(name='fisher_loss', tensor=fisher_loss)
        
        
with tf.name_scope('final_output'):
    y = tf.add(tf.matmul(hiddens[-1],weights[-1]),biases[-1]) # output layer
with tf.name_scope('xtropy_loss'):
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))
    tf.summary.scalar(name='xtropy_loss', tensor=cross_entropy_loss)
with tf.name_scope('total_loss'):
    total_loss = fisher_loss + cross_entropy_loss
    tf.summary.scalar(name='total_loss', tensor=total_loss)
with tf.name_scope('optimization'):
    train_step = tf.train.AdamOptimizer(0.1).minimize(total_loss)    
    
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar(name='accuracy', tensor=accuracy)
summ_op = tf.summary.merge_all()

Layer done 1


In [100]:
hiddens

[<tf.Tensor 'Relu:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden1:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden2:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden3:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden4:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden5:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden6:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden7:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden8:0' shape=(?, 600) dtype=float32>,
 <tf.Tensor 'hidden9:0' shape=(?, 600) dtype=float32>]

In [128]:
from sklearn import datasets
from keras.utils import to_categorical
#n_samples = 2000
#feat, labels = datasets.make_classification(n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1, n_samples=n_samples)
#feat, labels = datasets.make_moons(n_samples=n_samples, noise=0.05)
#feat, labels = datasets.make_blobs(n_samples=n_samples)
#print(feat.shape, labels.shape)
from tensorflow.examples.tutorials.mnist import input_data
# Load the MNIST data set
mnist_data = input_data.read_data_sets("./mnist", one_hot=True)
#labels = to_categorical(labels, num_classes=2)
#print(feat.shape, labels.shape)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter('./fishnetlog', sess.graph)
for i in range(2000):
    batch_x, batch_y = mnist_data.train.next_batch(100)
    _, summar, accu = sess.run([train_step, summ_op, accuracy], feed_dict={x: batch_x, y_: batch_y})
    writer.add_summary(summar, i)
    if i%100 == 0:
        print(accu)
temp_batch_x, temp_batch_y = mnist_data.train.next_batch(1000)
    #print(accu)
    #model.train_step.run(feed_dict={x: feat, y_: labels})
    #print(sess.run([accuracy], feed_dict={x: feat, y_: labels}))
activations = []
for i in hiddens:
    activations.append(sess.run(i, feed_dict={x:temp_batch_x}))
#morphs(activations, np.argmax(temp_batch_y, axis=1), skip=1, last_layer=True)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
0.09
0.05
0.11
0.11
0.11
0.16
0.1
0.09
0.06
0.11
0.09
0.06
0.1
0.1
0.06
0.07
0.08
0.1
0.08
0.13


In [125]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

def pca_morphs(activations, labels, skip=2, last_layer=False):
    trace_list = []
    n_rows = 1 if ((len(activations)) < 10) else -(-len(activations)//10)
   
    fig = tools.make_subplots(rows=n_rows, cols=(10))
    last_dim = (2 if last_layer==False else 0)
    for i in range(1, len(activations)-last_dim, skip):
        print(i)
        pca = PCA(n_components=2)
        m = pca.fit_transform(activations[i-1])
        print(m.shape)
        fig.append_trace(go.Scatter(x=m[:,0], y=m[:,1] , mode='markers', marker=dict(color=labels), text=labels), -(-i//10), (10 if i%10==0 else i%10))
    fig['layout'].update(height=n_rows*300)
    plot(fig)

In [129]:
activations = []
for i in hiddens:
    activations.append(sess.run(i, feed_dict={x:temp_batch_x}))
activations.append(sess.run(y, feed_dict={x:temp_batch_x}))
print(len(activations))
print(activations[0].shape)
pca_morphs(activations, np.argmax(temp_batch_y, axis=1), skip=1, last_layer=True)

4
(1000, 600)
This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]

1
(1000, 2)
2
(1000, 2)
3
(1000, 2)


/usr/local/lib/python3.6/site-packages/sklearn/decomposition/pca.py:505: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/pca.py:505: RuntimeWarning:

divide by zero encountered in true_divide



In [21]:
from keras.utils import to_categorical
from sklearn import datasets
n_samples = 2000
feat, labels = datasets.make_moons(n_samples=n_samples, noise=0.05)
print(feat.shape, labels.shape)
labels = to_categorical(labels, num_classes=2)
print(feat.shape, labels.shape)

(2000, 2) (2000,)
(2000, 2) (2000, 2)


In [32]:
print(len(activations))
print(activations[0].shape)
print(activations[1].shape)
print(activations[2].shape)
print(activations[3].shape)

4
(1000, 600)
(1000, 600)
(1000, 600)
(1000, 600)


In [40]:
len(activations)
print(activations[0].shape)
#morphs(activations, np.argmax(labels, axis=1), skip=1, last_layer=True)

(1000, 600)


In [141]:
np.linspace(1,100, n_layers)

array([  1.  ,  25.75,  50.5 ,  75.25, 100.  ])